**Multivariate Outlier Detection**

We are now going to look at multivariate outlier detection. What we mean by multivariate outlier detection is the use of more than one variable to identify an outlier. In univariate outlier detection we looked for individual outliers in a single variable. These univariate outliers may not show up if there are unusual combinations between 2 or more variables. This is where multivatriate outlier detiection comes in. The same principles as mentioned  in univariate outlier detection can be applied with multivariate outliers. In otherwords don't exclude particular rows of data unless you scientific evidence to support this.</br>
</br>


There are numerous approaches to multivariate outlier detection, some of which are outlined below:


*   Bivariate Charts
> * Based on bivariate Normal assumptions
*   Depth Based control charts (nonparameteric)
> * Map n-dimensional data to one dimension using for example Mahalanobis
> * Build Control Charts for depth

* Mutiscale process control wavelets
> * Detects abnormalities at multiple scales as large wavelett coefficients
> * Useful for data with hetroscedasticity
> * Used in chemical process control

In this section we will specifically focus on the depth based control charts as it is a reasonably reboust process and is quite quick to set-up. The Multiscale process are beyond this material at this stage.

We will need the following three python libaries.

In [ ]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
housing.DESCR
data = housing.data
target = housing.target
print(housing.feature_names)
x = data
y = target

In [ ]:
!pip install pyspc
!pip install matplotlib
!pip install spm1d


Now lets load the housing housing data that we used in the univariate approach.

In [ ]:
#from sklearn.datasets import load_housing


from sklearn.metrics import DistanceMetric as DM
import pandas as pd
housing = data


x = data
y = target
#create the dataframe
housing_df = pd.DataFrame(housing)
#housing_df.columns = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT']
#housing_df['Y']=pd.Series(y)
housing_df


We need to be able to calcualte multivariate distances to tells us which points are outliers. This is where were the library [*scipy.spatial*](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.mahalanobis.html)  comes in. This library calculates the Mahalanobis distance and then compares this to values for a hotellings t squared distribution to deterimine if they are outliers. Its like comparing a single univariate variable to a t-distribution. Have a look at this [link](https://en.wikipedia.org/wiki/Hotelling%27s_T-squared_distribution) to understand this better. The code below is an example for the housing dataset of how to calculate the critical Hotellings T Squared value for the dataset. So any values whoes distance is greater than our critical value will be considered outliers.


In [ ]:
from scipy.stats import f

x=housing_df.values.tolist()
#print(x))
n=len(x)
k=len(x[0])
vals = f.ppf([0.99], n, k)*(n*k)/((n-k)+1)
print(vals)
rv = f(len(x), len(x[0]))

The final piece of code tries to put all this together. We have found our crtical values by calcualting the Hotellings T-Squared from n rows with k columns. We then calcualte the Mahalanobis distance for each row from the overall multivariate mean $\bar{X}$ (xbar). Use this distance with the covariance matrix to calculate individual Hotelling T Squared statistics and  then compare each one to the critical Hotellings T Squared values. You will notice that we have used the covariance matrix to helps us standarised the data using *pandas.cov*. There will be small differences between the chart above and the one below as we do not have the exact calculations from *pyspc* but by using the code below we can identify the outlier points.

In [ ]:
from  scipy.spatial import distance
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import f
import pandas as pd
#print(housing_df.cov())
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
#print(housing)
print(housing.feature_names)

In [ ]:
from  scipy.spatial import distance
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import f
#print(housing_df.cov())
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
print(housing)
housing.DESCR
data = housing.data
target = housing.target
print(housing.feature_names)
x = data
y = target

inv_cov = pd.DataFrame(np.linalg.pinv(housing_df.cov().values), housing_df.cov().columns, housing_df.cov().index).values.tolist()
x=housing_df.values.tolist()
xbar=housing_df.mean().tolist()

n=len(x)
k=len(x[0])
## Convert f value to hotellings critical value at 1% level
##https://en.wikipedia.org/wiki/Hotelling%27s_T-squared_distribution

Hotvalue = f.ppf([0.99], n, k)*(n*k)/((n-k)+1)
print(Hotvalue)

## calculating hoteling test statistic for each row in housing dataset

for i in range(0,len(x)):

  housing_df.loc[housing_df.index==i,'hotelling']=(n*k)*distance.mahalanobis(x[i], xbar, inv_cov)**2/(k*(n-k))
  housing_df.loc[housing_df.index==i,'critical value']=Hotvalue

x=np.array(housing_df.index.tolist())
y1=np.array(housing_df['hotelling'])
f = plt.figure()

ax = f.add_subplot(111)

plt.plot(x, y1)
plt.axhline(y=housing_df['hotelling'].mean())
plt.axhline(y=Hotvalue,color='r')
#plt.axhline(y=housing_df['hotelling'].mean()+3*housing_df['hotelling'].std(),color='r')
#plt.axhline(y=housing_df['hotelling'].mean()-3*housing_df['hotelling'].std(),color='r')

plt.title('Multivariate Charts  ', fontsize=8)
print(housing_df.loc[housing_df['hotelling']>housing_df['critical value']])

You should be able to use this depth based approach to help you identify multivariate outliers. It is worth looking at this [article](https://onlinelibrary.wiley.com/doi/full/10.1002/cem.2763) from Richard Brereton if you want more detail. The code I have developed above is based on this paper.</br></br>
Finally, play with the data and test out for yourself if you can find or identify new outliers that you may create. Share your conclusions amongst the class.
